# Prediction on OpenBCI data
___

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import joblib
import numpy as np

from sklearn.metrics import (accuracy_score,
                             confusion_matrix,
                             plot_confusion_matrix,
                             classification_report,
                             f1_score,
                             cohen_kappa_score,
                             log_loss)

from scipy.signal import medfilt

from hmmlearn.hmm import MultinomialHMM
from constants import (SLEEP_STAGES_VALUES, N_STAGES, EPOCH_DURATION)
from models.model_utils import print_hypnogram

In [ ]:
USE_CONTINUOUS = False

if USE_CONTINUOUS:
    X = np.load("./data/X_openbci-age-continuous.npy", allow_pickle=True)
    y = np.load("./data/y_openbci-age-continuous.npy", allow_pickle=True)
else:  
    X = np.load("data/X_openbci.npy", allow_pickle=True)
    y = np.load("data/y_openbci.npy", allow_pickle=True)

X = np.vstack(X)
y = np.hstack(y)

print(X.shape)
print(y.shape)

In [ ]:
print(X[:,1])

## Load classifier
___

In [ ]:
MODELS_DIR = "trained_model"

if USE_CONTINUOUS:
    classifier = joblib.load(f"{MODELS_DIR}/classifier_RF_continous_age.joblib")
else:
    classifier = joblib.load(f"{MODELS_DIR}/classifier_RF.joblib")


## Prediction
___

In [ ]:
%%time

y_pred = classifier.predict(X)

In [ ]:
print(confusion_matrix(y, y_pred))

print(classification_report(y, y_pred, target_names=SLEEP_STAGES_VALUES.keys()))

print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y, y_pred))

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
print_hypnogram([y_pred, y], labels=["Classifier", "Alexandra"], subject=0, night=0)

## Results from models
____

### 1. RF
____

#### a) Not tuned
Hyperparameters configuration: `(n_estimators=100, others=default)`

```
              precision    recall  f1-score   support

           W       0.90      0.95      0.92       379
          N1       0.00      0.00      0.00        58
          N2       0.78      0.77      0.78       353
          N3       0.76      0.95      0.85       242
         REM       0.79      0.72      0.75       200

    accuracy                           0.82      1232
   macro avg       0.65      0.68      0.66      1232
weighted avg       0.78      0.82      0.80      1232

Agreement score (Cohen Kappa):  0.7545615539827596
```

#### b) Tuned version 1
____

Hyperparameters configuration: `(n_estimators=592, others=default)`

```
               precision    recall  f1-score   support

           W       0.90      0.95      0.92       379
          N1       0.00      0.00      0.00        58
          N2       0.76      0.76      0.76       353
          N3       0.76      0.95      0.85       242
         REM       0.79      0.69      0.74       200

    accuracy                           0.81      1232
   macro avg       0.64      0.67      0.65      1232
weighted avg       0.77      0.81      0.79      1232

Agreement score (Cohen Kappa):  0.743444308164724


```

#### c) Tuned version 2
___

Hyperparameters configuration: `(n_estimators=592, max_depth=24, others=default)`

```
              precision    recall  f1-score   support

           W       0.90      0.94      0.92       379
          N1       0.00      0.00      0.00        58
          N2       0.73      0.75      0.74       353
          N3       0.75      0.95      0.84       242
         REM       0.77      0.66      0.71       200

    accuracy                           0.80      1232
   macro avg       0.63      0.66      0.64      1232
weighted avg       0.76      0.80      0.77      1232

Agreement score (Cohen Kappa):  0.725949621846069
```

## Median filter
___

In [ ]:
KERNEL_SIZE=3

y_medfilt_pred = medfilt(y_pred, kernel_size=KERNEL_SIZE)

In [ ]:
print(confusion_matrix(y, y_medfilt_pred))

print(classification_report(y, y_medfilt_pred, target_names=SLEEP_STAGES_VALUES.keys()))

print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y, y_medfilt_pred))

print_hypnogram([y_medfilt_pred, y], labels=[ "Classifier (Medfilter)", "Alexandra"], subject=0, night=0)

## Hidden Markov Model
___

In [ ]:
hmm_model = MultinomialHMM(n_components=N_STAGES)


hmm_model.emissionprob_ = np.load(f"{MODELS_DIR}/HMM_emissionprob.npy", allow_pickle=True)
hmm_model.startprob_ = np.load(f"{MODELS_DIR}/HMM_startprob.npy", allow_pickle=True)
hmm_model.transmat_ = np.load(f"{MODELS_DIR}/HMM_transmat.npy", allow_pickle=True)

y_hmm_pred = hmm_model.predict(y_pred.reshape(-1, 1))

In [ ]:
print(confusion_matrix(y, y_hmm_pred))

print(classification_report(y, y_hmm_pred, target_names=SLEEP_STAGES_VALUES.keys()))

print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y, y_hmm_pred))

print_hypnogram([y_hmm_pred, y], labels=[ "Classifier (HMM)", "Alexandra"], subject=0, night=0)